# Викишоп

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from concurrent.futures import ProcessPoolExecutor
from sklearn.pipeline import Pipeline
import spacy
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import lightgbm as lgb
from tqdm import tqdm

In [3]:
# Загрузка данных
data = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv'
    )

In [5]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

In [6]:
display(data.head())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
# Лемматизация
nlp = spacy.load('en_core_web_sm', disable=[ 'parser',  'ner'])

def lemmatize_clear_text_spacy(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.is_alpha])

In [8]:
# Используем ProcessPoolExecutor для параллельной обработки
with ProcessPoolExecutor() as executor:
    data['clear_text'] = list(
        executor.map(lemmatize_clear_text_spacy, data['text'])
    )

## Обучение

In [11]:
# Определяем данные для обучения
X = data['clear_text']
y = data['toxic']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.1, 
    random_state=42
)

# Создаем TF-IDF векторизатор с исключением стоп-слов
stopwords = list(nltk_stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(
    stop_words=stopwords,
    max_df=0.8,
    min_df=5,
    ngram_range=(1, 3)
)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [14]:
# Настраиваем модели
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced', C=7.5)

svc = LinearSVC(C=10, max_iter=10000, class_weight='balanced')

pos_weight = len(data[data['toxic']==0]) / len(data[data['toxic']==1])
lgbm = lgb.LGBMClassifier(
    objective='binary',
    n_estimators=500,
    learning_rate=0.2,
    max_depth=-1,
    force_col_wise=True,
    scale_pos_weight=pos_weight,
    verbose=0
)

# Список моделей
models = [
    ('Logistic Regression', log_reg),
    ('Linear SVC', svc),
    ('LightGBM', lgbm)
]

# Создаем список для хранения результатов
results = []

for name, model in models:
    scores = cross_val_score(model, X_train_tfidf, y_train, cv=5, scoring='f1')
    results.append({
        'Model': name,
        'F1_Mean': scores.mean(),
    })

# Преобразуем в DataFrame
results_df = pd.DataFrame(results).set_index('Model')

display(results_df)

,F1_Mean
Model,
Logistic Regression,0.773571
Linear SVC,0.740643
LightGBM,0.763342


Лидирует простая LogisticRegression, проверяем на тесте

In [15]:
# Предсказания
log_reg.fit(X_train_tfidf, y_train)
predict = log_reg.predict(X_test_tfidf)

# Оценка по F1-score на трейне
scores = f1_score(y_test, predict)
print(f"F1-score: {scores:.2f}")

F1-score: 0.78


## Выводы

В данном проекте мы обработали датасет с твитами, использовали лемматизацию для очистки текста. Так же использовали ProcessPoolExecutor для ускоренной лемматизации, применили TF-IDF векторизатор с набором стоп-слов для английского языка. В дальнейшем обучили LogisticRegression, Linear SVC и LightGBM. Лучшая оказалась LogisticRegression с выходом метрики F1 на тестовых данных в 0.75